In [ ]:
# Instalando e atualizando as bibliotecas necessárias
!pip install pyDOE
!pip install -U seaborn

In [1]:
# Importando RASD Toolbox
from RASD_TOOLBOX import *
# Importando Numpy toolbox
import numpy as np
from VIGA_METALICA_VERIFICA import *
from FINITO_FEM_TOOLBOX import *

In [7]:
ELEMENTO_ANALISADO = 2

P1 = ['NORMAL', 200E9, 20E9] #E MPa              
P2 = ['NORMAL', 250E6, 25E6] #Fy MPa               
P3 = ['NORMAL', 10000, 1000] #Fy N
P4 = ['NORMAL', 1, 0.05] #Incerteza de Modelo adimensional 
P5 = ['NORMAL', 1, 0.05] #Incerteza de Modelo adimensional 
POP = [100]


#carga G, carga Q, tensao escoamento (Fy) e modulo de elasticidade (ES)
#carta_total = carga G + carga Q

#POP = np.arange(10,1000000, 10000).tolist() # (x,y,Z); x=pop inicial; y=pop final; z=variacao pop

SETUP = {'N_REP': len(POP),
        'POP': POP,
        'N_G': 3,
        'D': 5, #variables
        'VARS': [P1, P2, P3, P4, P5],
        'MODEL': 'MCS'}

In [8]:
def OBJ(X):
    
    E = X[0]
    FY = X[1]
    F1 = X[2]
    M1 = X[3]
    M2 = X[4]
   
    alfa = 0.001 #tranformar em metro
    beta = 0.000001
    gamma = 0.00000001
    
    VIGA = {'H_W': 139 * alfa, #mm            
        'T_W': 5.8 * alfa, #mm
        'B_F': 102 * alfa, #mm
        'T_F': 7.1 * alfa, #mm
        'W_C': 1, #mm ?
        'W_T': 1, #mm ?
        'PARAMETRO_PERFIL': 'DUPLA SIMETRIA', #"DUPLA SIMETRIA, MONO SIMETRIA "
        'TIPO_PERFIL': 'LAMINADO', #"SOLDADO, LAMINADO "
        'GAMMA_A1': 1.1,
        'Z': 139.4 * beta, #cm3
        'INERCIA':939 * gamma, #cm4   
        'E_S': E , #*100 para transformar de MPa para kn/cm2
        'F_Y': FY} #*100 para transformar de MPa para kn/cm2
    
    FRAME_00 = {
            "TYPE_ELEMENT": 0,
            "TYPE_SOLUTION": 0,
            "N_NODES": 8,
            "N_MATERIALS": 1,
            "N_SECTIONS": 1,
            "N_ELEMENTS": 9,
            "N_DOFPRESCRIPTIONS": 6,
            "N_DOFLOADED": 4,
            "N_DOFSPRINGS": 0,
            "COORDINATES": #X,Y
            np.array([
            [0.0,0.0],
            [5.25,0.0],
            [0.0,3.0],
            [5.25,3.0],
            [0.0,6.0],
            [5.25,6.0],
            [0.0,9.0],
            [5.25,9.0]]),

            "ELEMENTS": #NODE 1,NODE 2,MATERIAL ID,SECTION ID,HINGE ID NODE 1,HINGE ID NODE 2
            np.array([
            [0,2,0,0,0,0],
            [1,3,0,0,0,0],
            [2,3,0,0,0,0],
            [2,4,0,0,0,0],
            [3,5,0,0,0,0],
            [4,5,0,0,0,0],
            [4,6,0,0,0,0],
            [5,7,0,0,0,0],
            [6,7,0,0,0,0]]),

            "MATERIALS": #YOUNG,POISSON,DENSITY,THERMAL COEFFICIENT
            #np.array([[E, 0.30, 1, 1E-8]]),                              #ES(young) deve vir do perfil, ver unidades
            np.array([[200E9, 0.30, 1, 1E-8]]), 
            "SECTIONS": #AREA,INERTIA 1,INERTIA 2,X GC,Y GC
            #np.array([[0.02, VIGA['INERCIA']*E-8, VIGA['INERCIA']*E-8, 0.05, 0.1]]), #Inercia *E-8 para conveter de cm4 para m4
             np.array([[0.02,939E-9,939E-9,0.05,0.1]]),  #Inercia *E-8 para conveter de cm4 para m4
            "PRESCRIBED DISPLACEMENTS": #NODE,DIRECTION(X=0,Y=1,Z=2),VALUE
            np.array([[0,0,0],
            [0,1,0],
            [0,2,0],
            [1,0,0],
            [1,1,0],
            [1,2,0]]),
            "ELEMENT LOADS": None,
            "NODAL LOADS":  #NODE,DIRECTION(X=0,Y=1,Z=2),VALUE
            np.array([
            [2,0, F1, 0],                                           
            [4,0, F1, 0],
            [6,0, F1, 0],
            [6,1,-F1, 0],
            [7,1,-F1, 0]]),
            "SPRINGS": None}
    
    FRAME_00_RESULTS = MEF1D(DICTIONARY = FRAME_00)   
    ELEMENT = pd.DataFrame(data = FRAME_00_RESULTS[ELEMENTO_ANALISADO])
    M_SD = max(ELEMENT['M'].max(),abs(ELEMENT['M'].min()))
    V_SD = max(ELEMENT['V'].max(),abs(ELEMENT['V'].min()))
    N_SD = max(ELEMENT['N'].max(),abs(ELEMENT['N'].min()))
    D_SD = min(ELEMENT['UX'].max(),abs(ELEMENT['UX'].min()))
    L_MAX = ELEMENT['X'].max()
       
    ESFORCOS = {'M_SD': M_SD,
                'V_SD': V_SD,
                'D_SD': D_SD,
                'L_MAX': L_MAX}
    
    R_0, S_0 = VERIFICACAO_VIGA_METALICA_MOMENTO_FLETOR(VIGA, ESFORCOS) 
    G_0 = -M1 * R_0 + M2 * S_0

    R_1, S_1 = VERIFICACAO_VIGA_METALICA_ESFORCO_CORTANTE(VIGA, ESFORCOS)
    G_1 = -M1 * R_1 + M2 * S_1
    
    R_2, S_2 = VERIFICACAO_VIGA_METALICA_DEFORMACAO(VIGA, ESFORCOS)
    G_2 = -M1 * R_2 + M2 * S_2
    
    R = [R_0, R_1, R_2] 
    S = [S_0, S_1, S_2] 
    G = [G_0, G_1, G_2]
    
    #ver unidades, deslocamentos, ver sinal deslocamento .abs
    return R, S, G

In [9]:
RESULTS_TEST = RASD_STOCHASTIC(SETUP, OBJ)

In [10]:
RESULTS_TEST[0]['TOTAL RESULTS']

,X_0,X_1,X_2,X_3,X_4,R_0,R_1,R_2,S_0,S_1,S_2,G_0,G_1,G_2,I_0,I_1,I_2
0,1.885781e+11,2.488147e+08,10122.867141,0.952163,0.989509,31531.608462,109415.133752,0.015,27934.897982,10641.845004,0.003771,-2381.381737,-93650.791985,-0.010551,0.0,0.0,0.0
1,1.821325e+11,2.122337e+08,12169.531918,0.992442,1.039338,26895.799206,93328.809124,0.015,33584.249510,12793.974694,0.004534,8212.855897,-79326.180553,-0.010174,1.0,0.0,0.0
2,1.639841e+11,2.476224e+08,8469.783735,1.003874,1.026172,31380.508259,108890.813880,0.015,23372.915531,8903.950339,0.003156,-7517.458560,-100175.704158,-0.011820,0.0,0.0,0.0
3,2.223538e+11,2.489769e+08,9243.156481,0.970029,1.068491,31552.162585,109486.456856,0.015,25509.016207,9717.701380,0.003444,-3350.342289,-95821.722600,-0.010871,0.0,0.0,0.0
4,1.996622e+11,2.562406e+08,9310.106226,1.010931,1.164955,32472.675015,112680.648194,0.015,25693.924146,9788.142362,0.003469,-2895.367402,-102509.576326,-0.011123,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,2.065947e+11,2.339470e+08,10333.373082,0.998133,0.935855,29647.468136,102877.139791,0.015,28517.220000,10863.681528,0.003850,-2904.110280,-92518.198372,-0.011369,0.0,0.0,0.0
96,1.902871e+11,2.469873e+08,10236.863818,0.925398,1.070523,31300.024619,108611.534497,0.015,28249.517461,10761.699808,0.003814,1276.768859,-88988.289052,-0.009798,1.0,0.0,0.0
97,2.217952e+11,2.554492e+08,8660.854142,0.999320,0.917832,32372.379141,112332.620072,0.015,23900.041150,9104.759705,0.003227,-10414.156815,-103899.631132,-0.012028,0.0,0.0,0.0
98,2.048716e+11,2.748379e+08,9272.949728,0.983040,1.068226,34829.461939,120858.732633,0.015,25589.050986,9748.190760,0.003455,-6903.891713,-108395.761426,-0.011055,0.0,0.0,0.0


In [ ]:
DADOS = RESULTS_TEST[0]['TOTAL RESULTS']

In [ ]:
DADOS['I_0'].sum()

In [ ]:
DADOS = RESULTS_TEST[len(POP)-1]['TOTAL RESULTS']

In [ ]:
OPCOES_GRAFICAS = {'NAME': 'WANDER',
                'WIDTH': 0.40, 
                'HEIGHT': 0.20, 
                'X AXIS SIZE': 20,
                'Y AXIS SIZE': 20,
                'AXISES COLOR': '#000000',
                'X AXIS LABEL': '$x_0$ - $P_X (lb)$',
                'LABELS SIZE': 16,
                'LABELS COLOR': '#000000',  
                'CHART COLOR': '#FEB625',
                'KDE': False,
                'DPI': 600, 
                'BINS' : 20,
                'EXTENSION': '.svg'}
# RESULTS X_0 VARIABLE
OPCOES_DADOS = {'DATASET': DADOS, 'COLUMN': 'X_0'}      
# CALL PLOT
RASD_PLOT_1(OPCOES_DADOS, OPCOES_GRAFICAS)

In [ ]:
OPCOES_GRAFICAS = {'NAME': 'WANDER',
                'EXTENSION': '.svg',
                'DPI': 600,
                'WIDTH': 0.300, 
                'HEIGHT': 0.150,              
                'X DATA': 'S_0',
                'Y DATA': 'R_0',
                'X AXIS SIZE': 16,
                'Y AXIS SIZE': 16,
                'AXISES COLOR': '#000000',
                'X AXIS LABEL': '$S_0$',
                'Y AXIS LABEL': '$R_0$',
                'LABELS SIZE': 18,
                'LABELS COLOR': '#000000',
                'LOC LEGEND': 'lower right',
                'TITLE LEGEND': 'Failure index ($I$):'}
# RESULTS
OPCOES_DADOS = {'DATASET': DADOS, 'X DATA': 'S_0', 'Y DATA': 'R_0', 'HUE VALUE': 'I_0'}    
# CALL PLOT
RASD_PLOT_2(OPCOES_DADOS, OPCOES_GRAFICAS)

In [ ]:
OPCOES_GRAFICAS = {'NAME': 'WANDER',
                'EXTENSION': '.svg',
                'DPI': 600,
                'WIDTH': 0.20, 
                'HEIGHT': 0.10,              
                'X DATA': 'S_0',
                'Y DATA': 'R_0',
                'X AXIS SIZE': 20,
                'Y AXIS SIZE': 20,
                'AXISES COLOR': '#000000',
                'X AXIS LABEL': '$S_0$',
                'Y AXIS LABEL': '$R_0$',
                'LABELS SIZE': 16,
                'LABELS COLOR': '#000000',
                'G VALUE': 'G_0',
                'TRANSPARENCY': 0.8,
                'COLOR MAP': 'viridis'}


# RESULTS
OPCOES_DADOS = {'DATASET': DADOS, 'X DATA': 'S_0', 'Y DATA': 'R_0', 'G VALUE': 'G_0'}       

# CALL PLOT
RASD_PLOT_3(OPCOES_DADOS, OPCOES_GRAFICAS)

In [ ]:
OPCOES_GRAFICAS = {'NAME': 'WANDER',
                    'EXTENSION': '.svg',
                    'DPI': 600,
                    'WIDTH': 0.20,
                    'HEIGHT': 0.10,
                    'X DATA': 'S_0',
                    'Y DATA': 'R_0',
                    'X AXIS SIZE': 20,
                    'Y AXIS SIZE': 20,
                    'AXISES COLOR': '#000000',
                    'X AXIS LABEL': '$G_0 + G_0$',
                    'Y AXIS LABEL': '$SOMATÓRIO$',
                    'LABELS SIZE': 16,
                    'LABELS COLOR': '#000000',
                    'C VALUE': 'G_0',
                    'TRANSPARENCY': 0.8,
                    'COLOR MAP': 'viridis',
                    'BINS': '50',
                    'ALPHA': '0.5'}

# RESULTS
OPCOES_DADOS = {'DATASET': DADOS, 'X DATA': 'S_0', 'Y DATA': 'R_0', 'G VALUE': 'G_0'}       

# CALL PLOT
RASD_PLOT_4(OPCOES_DADOS, OPCOES_GRAFICAS)

In [ ]:
OPCOES_GRAFICAS = {'NAME': 'WANDER',
                    'EXTENSION': '.svg',
                    'DPI': 600,
                    'WIDTH': 0.20,
                    'HEIGHT': 0.10,
                    'X AXIS SIZE': 20,
                    'Y AXIS SIZE': 20,
                    'AXISES COLOR': '#000000',
                    'X AXIS LABEL': 'Número de Simulações (ns)',
                    'Y AXIS LABEL': 'Probabilidade de Falha',
                    'LABELS SIZE': 16,
                    'LABELS COLOR': '#000000',
                    'CHART COLOR': 'black',
                    'POPULATION' : POP,
                    'TYPE' : 'pf'}

# RESULTS
OPCOES_DADOS = {'DATASET': RESULTS_TEST}       

# CALL PLOT
RASD_PLOT_5(OPCOES_DADOS, OPCOES_GRAFICAS)

In [ ]:
OPCOES_GRAFICAS = {'NAME': 'WANDER',
                    'EXTENSION': '.svg',
                    'DPI': 600,
                    'WIDTH': 0.20,
                    'HEIGHT': 0.10,
                    'X AXIS SIZE': 20,
                    'Y AXIS SIZE': 20,
                    'AXISES COLOR': '#000000',
                    'X AXIS LABEL': 'Número de Simulações (ns)',
                    'Y AXIS LABEL': 'Beta',
                    'LABELS SIZE': 16,
                    'LABELS COLOR': '#000000',
                    'CHART COLOR': 'black',
                    'POPULATION' : POP,
                    'TYPE' : 'beta'}

# RESULTS
OPCOES_DADOS = {'DATASET': RESULTS_TEST}       

# CALL PLOT
RASD_PLOT_5(OPCOES_DADOS, OPCOES_GRAFICAS)

In [ ]:
10E-